In [155]:
from numpy import exp, array, random, dot
import numpy as np
from scipy import misc
import skimage
import math
from __future__ import with_statement

In [156]:
arr1 = np.divide(misc.imread('save_save_america01.jpg'),255)
arr1r = arr1[:,:,0]
arr1g = arr1[:,:,1]
arr1b = arr1[:,:,2]
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [157]:
filterNum = 1
filterSize = 10
layerNum = 6
epochs = 10
filters = []

In [165]:
def setFilters():
    global filters
    filters = 0.01*random.random((filterNum*layerNum, filterSize, filterSize))-0.005
setFilters()
print(filters)

[[[ -5.93052153e-04  -2.91177303e-03   3.39094053e-03   2.85056236e-03
    -1.08867909e-03   1.18502118e-04   2.98562097e-05   1.38495136e-03
     2.23978703e-03  -3.23242058e-03]
  [ -3.06792795e-03   6.73534229e-04   4.58395136e-03  -2.89574408e-03
    -4.64162426e-03   3.53636661e-03  -4.26308775e-03  -4.89645561e-03
    -1.93524232e-03   2.51124167e-04]
  [  3.35726808e-03  -1.93481551e-04   4.50749539e-03  -1.48314046e-04
     3.40132089e-03   3.40517172e-03  -2.73588155e-03  -3.96417370e-04
     2.41313701e-03  -1.05287598e-03]
  [ -4.33255518e-03   2.25854290e-03  -2.01714576e-03   8.43654275e-04
     4.93257413e-04   2.79147862e-03   7.17869990e-04  -3.37451618e-03
    -2.26329486e-03   4.69584267e-03]
  [ -1.17926800e-03  -3.91177002e-03   3.08731160e-03  -1.99480468e-03
    -2.75253462e-03  -2.95161122e-03   2.90334874e-03   6.52373780e-04
     3.22945802e-03  -3.30067149e-03]
  [  1.75737494e-03  -8.56351532e-04  -1.80792827e-03   3.25192347e-03
    -4.61850664e-03   4.51532

In [180]:
def activation(x, deriv=False, leak=-0.01, arr=False):
    if deriv:
        return (x > 0)
    if arr:
        new_x = x * (x > 0)
        new_x[x==0] = leak
        return new_x
    new_x = x * (x > 0)
    if new_x==0:
        new_x=leak
    return new_x
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
def apply(filterArr, img, row, col):
    value = 0
    halfFil = int(filterSize/2)
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0))
    for y in range(len(filterArr)):
        for x in range(len(filterArr[0])):
            value += padImg[y+row-halfFil][x+col-halfFil]*filterArr[y][x]
    if sumNoNeg(filterArr)==0:
        return 0
    return value
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)
print(apply(filters[4],arr1r,0,0))

0.0


In [167]:
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])):
            new_img[y][x] = activation(apply(filterArr,img,y,x))
    return new_img
# print(len(([[0]*(len(arr1r))]*(len(arr1r[0])))[0]))
# print(len(arr1r[0]))
# #print([[0]*(len(arr1r)-filterSize)]*(len(arr1r[0])-filterSize))
# print(len(arr1r))
# print(len(arr1r[0]))
# newArr = applyFilters(arr1r, np.zeros((len(arr1r), len(arr1r[0]))), filters[0])
# print(len(newArr))
# print(len(newArr[0]))

In [182]:
def forward(img):
    outputLayers = [[img]]
    for num in range(layerNum):
        outputLayers.append([])
        for fil in range(filterNum):
            newImage = np.zeros((len(outputLayers[num][fil]), len(outputLayers[num][fil][0])))
            applyFilters(outputLayers[num][fil], newImage, filters[filterNum*num + fil])
            print(filterNum*num + fil)
            print(newImage)
            outputLayers[num+1].append(newImage)
    outputLayers.append(softmax(array(outputLayers[len(outputLayers)-1]).flatten()))
    return outputLayers
out = forward(arr1r)
print(out)

0
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02   6.03631699e+27   6.98951308e+33 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02   1.68207184e+32 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 ..., 
 [ -1.00000000e-02   8.37258897e+32   8.80727462e+35 ...,   6.73431303e+38
    6.73454684e+38   6.57121269e+38]
 [ -1.00000000e-02   8.40063907e+32  -1.00000000e-02 ...,   6.86392492e+38
    6.86076837e+38   6.86055063e+38]
 [ -1.00000000e-02   8.76954640e+32  -1.00000000e-02 ...,   2.55401058e+38
    2.79106513e+38   2.22236547e+38]]
1
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,   5.65886666e+31
    5.65886666e+31   5.65886666e+31]
 [ -1.00000000e-02  -1.00000000e-02   6.56493049e+53 ...,   5.20573101e+31
    5.20573101e+3

FloatingPointError: overflow encountered in exp

In [ ]:
def maxPool(size, img):
    new_img = np.zeros((int(len(img)/size), int(len(img[0])/size)))
    pad_img = np.pad(img, [(0, size-len(img)%size),(0, size-len(img[0])%size)], "constant", constant_values=(0))
    for x in range(math.ceil(len(img)/size)-1):
        for y in range(math.ceil(len(img[0])/size)-1):
            new_img[x][y] = np.amax(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
            print(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
    return new_img
print(maxPool(3, out[2]))

In [177]:
error = 1-np.mean(out[len(out)-1])
# def back(img, startlayer, error):
#     print(startlayer)
#     newout = np.zeros((len(out[startlayer-1]), len(out[startlayer-1][0])))
#     if startlayer == -1:
#         return out[0]
#     delta = applyFilters(back(img, startlayer-1, error),
#                  newout,
#                  np.multiply(np.rot90(np.rot90(filters[startlayer])),activation(img,deriv=True)[:filterSize,:filterSize]))
#     filters[startlayer]=np.add(filters[startlayer],array(delta[:filterSize,:filterSize]))
#     print(startlayer)
#     return newout
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize))
    for x in range(filterNum):
        r = 0
        for row in filters[filterNum*layer+x-1]:
            c=0
            for w in row:
                new_grad[x][r][c]=-np.multiply(error*10,np.mean(np.multiply(np.multiply(
                    activation(array(out[layer-1]),deriv=True,arr=True),
                    w),
                    grad[x][r][c])))
#                 print(-np.divide(error,np.sum(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]))))
#                 print(new_grad[x][r][c])
                filters[filterNum*layer+x-1][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

6
5
4
3
2


FloatingPointError: overflow encountered in multiply

In [179]:
for x in range(epochs):
    out = forward(arr1r)
    error = 1-np.mean(out[len(out)-1])
    print(error)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))

0
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02   2.41711664e-05   2.95384986e-05 ...,   1.09906513e-03
    1.13950839e-03   7.90872850e-04]
 [ -1.00000000e-02  -1.00000000e-02   6.41706246e-05 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 ..., 
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]]
1
[[ -1.00000000e-02  -1.00000000e-02  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-02  -1.00000000e-02]
 [ -1.00000000e-02   2.63865919e-05  -1.00000000e-02 ...,   1.45131016e-04
    1.45131016e-04   1.45131016e-04]
 [ -1.00000000e-02   1.03771772e-05  -1.00000000e-02 ...,  -1.00000000e-02
   -1.00000000e-0

FloatingPointError: overflow encountered in multiply

In [178]:
setFilters()
np.seterr(divide='raise')
np.seterr(invalid='raise')
np.seterr(over='raise')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'ignore'}